In [1]:
import os

In [2]:
%pwd

'/home/blackitalian/ds_projects/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/blackitalian/ds_projects/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [5]:
# Add the parent directory of 'src' to the Python path
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2024-07-13 09:09:30.224622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 09:09:30.238497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 09:09:30.238530: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-13 09:09:30.248500: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-13 09:09:30.837522: W tensorflow/compiler/tf

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        # Recompile the model with a new optimizer instance
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [11]:
# Test loading and compiling the model
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    print("Model loaded and compiled successfully")
except Exception as e:
    print(e)

[2024-07-13 09:09:31,623: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-13 09:09:31,625: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-13 09:09:31,626: INFO: common: created directory at: artifacts]
[2024-07-13 09:09:31,627: INFO: common: created directory at: artifacts/training]


2024-07-13 09:09:31.824714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 09:09:31.848714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 09:09:31.848773: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 09:09:31.851969: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 09:09:31.852041: I external/local_xla/xla/stream_executor

[2024-07-13 09:09:32,424: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model loaded and compiled successfully


_device.cc:2019] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-07-13 09:09:32.033045: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 09:09:32.033069: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7537 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:09:00.0, compute capability: 8.6


In [12]:
# Test data generators
try:
    training.train_valid_generator()
    print("Data generators created successfully")
except Exception as e:
    print(e)


Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Data generators created successfully


In [13]:
# Simplified training loop
try:
    training.steps_per_epoch = training.train_generator.samples // training.train_generator.batch_size
    training.validation_steps = training.valid_generator.samples // training.valid_generator.batch_size

    training.model.fit(
        training.train_generator,
        epochs=1,  # Reduce the number of epochs
        steps_per_epoch=training.steps_per_epoch,
        validation_steps=training.validation_steps,
        validation_data=training.valid_generator
    )

    training.save_model(
        path=training.config.trained_model_path,
        model=training.model
    )
    print("Training completed successfully")
except Exception as e:
    print(e)


/home/blackitalian/miniconda3/envs/cancer/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1720876174.530368  802891 service.cc:145] XLA service 0x7fc72800bc90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720876174.530421  802891 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-13 09:09:34.549459: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-13 09:09:34.680630: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 

 3/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.5625 - loss: 1.3836

I0000 00:00:1720876177.783797  802891 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 207ms/step - accuracy: 0.5505 - loss: 1.4526 - val_accuracy: 0.7812 - val_loss: 0.4113
[2024-07-13 09:09:41,124: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
Training completed successfully


In [14]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()

    # Simplified training loop with reduced resource usage
    training.steps_per_epoch = training.train_generator.samples // training.train_generator.batch_size
    training.validation_steps = training.valid_generator.samples // training.valid_generator.batch_size

    training.model.fit(
        training.train_generator,
        epochs=1,  # Reduce the number of epochs
        steps_per_epoch=training.steps_per_epoch,
        validation_steps=training.validation_steps,
        validation_data=training.valid_generator
    )

    training.save_model(
        path=training.config.trained_model_path,
        model=training.model
    )
    print("Training completed successfully")
except Exception as e:
    print("Error in training model:", e)


[2024-07-13 09:09:41,263: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-13 09:09:41,265: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-13 09:09:41,266: INFO: common: created directory at: artifacts]
[2024-07-13 09:09:41,267: INFO: common: created directory at: artifacts/training]
[2024-07-13 09:09:41,379: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 131ms/step - accuracy: 0.5976 - loss: 0.9059 - val_accuracy: 1.0000 - val_loss: 0.0739
[2024-07-13 09:09:45,995: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`

In [15]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-07-13 09:09:46,129: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-13 09:09:46,131: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-13 09:09:46,133: INFO: common: created directory at: artifacts]
[2024-07-13 09:09:46,134: INFO: common: created directory at: artifacts/training]
[2024-07-13 09:09:46,246: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - accuracy: 0.5822 - loss: 0.9960 - val_accuracy: 0.9375 - val_loss: 0.1068
[2024-07-13 09:09:50,868: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`